In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import numpy as np
import faiss

In [3]:
particle_count = 10**5
particles = np.random.uniform(size=(particle_count, 3)).astype(dtype='float32')

In [4]:
# GPU
gpu_resources = faiss.StandardGpuResources()
gpu_device_id = 0

In [5]:
%%timeit
index_gpu = faiss.IndexFlatL2(particles.shape[1])
index_gpu = faiss.index_cpu_to_gpu(gpu_resources, gpu_device_id, index_gpu)
index_gpu.add(particles)
distances, indices = index_gpu.search(particles, 50)

1.12 s ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
# CPU
index_cpu = faiss.IndexHNSWFlat(particles.shape[1], 32)
# Parameters require empirical tuning
# index_flat.efConstruction = 40
index_cpu.efSearch = 100
index_cpu.add(particles)
distances, indices = index_cpu.search(particles, 50)

2.69 s ± 973 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# (indices[:, 0] == np.arange(len(points))).mean()